In [1]:
from perses.analysis.analysis import Analysis
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pymbar
%matplotlib inline
import os
import itertools
from tqdm import notebook as tqdm_notebook
import pandas as pd


from simtk.openmm import unit
from openmmtools.constants import kB
KT_KCALMOL = kB * 300 * unit.kelvin / unit.kilocalories_per_mole

In [19]:
# ignore seaborn displot warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [20]:
def subtract_offset(forward_work, reverse_work):

    print("--> subtracting offset")
    
    forward_work_offset = []
    for cycle in forward_work:
        forward_work_offset.append(np.array([val - cycle[0] for val in cycle[1:]]))
    forward_work_offset = np.array(forward_work_offset)

    reverse_work_offset = []
    for cycle in reverse_work:
        reverse_work_offset.append(np.array([val - cycle[0] for val in cycle[1:]]))
    reverse_work_offset = np.array(reverse_work_offset)
    
    return forward_work_offset, reverse_work_offset


def analyse(forward_accumulated, reverse_accumulated):
    
    print("--> computing dg, ddg")
    dg, ddg = pymbar.bar.BAR(forward_accumulated, reverse_accumulated)
    
    return dg, ddg


def plot_works(forward_work_offset,
               reverse_work_offset,
               dg,
               ddg,
               phase,
               mutation,
               title,
               save=False,
               output_dir=None):
    
    CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']
    
    # Plot work trajectories
    # TODO: automatically determine the x axis -> this is a bit of a hack at the moment
    print("--> plotting work trajs")
    
    for i, cycle in enumerate(forward_work_offset):
        
        x = [(j+1)*12.1e-4 for j in range(len(list(cycle)))]
        y = cycle
        if i==0:
            plt.plot(x, y, color=CB_color_cycle[0], label='forward')
        else:
            plt.plot(x, y, color=CB_color_cycle[0])
        
    for i, cycle in enumerate(reverse_work_offset):
        
        x = [(j+1)*12.1e-4 for j in range(len(list(cycle)))]
        y = -cycle
        if i==0:
            plt.plot(x, y, color=CB_color_cycle[1], label='reverse')
        else:
            plt.plot(x, y, color=CB_color_cycle[1])
        
    plt.xlabel("$t_{neq}$ (ns)")
    plt.ylabel("work (kT)")
    plt.title(f"{title} {phase}")
    plt.legend(loc='best')
    if save:
        if output_dir is not None:
            plt.savefig(os.path.join(output_dir, f"{mutation}_{phase}_work_traj.png"), dpi=500)
            print(f"--> saved to: {os.path.join(output_dir, f'{mutation}_{phase}_work_traj.png')}")
        else:
            print("--> No output_dir specified!")
    else:
        plt.show()
    plt.clf()
    
    # Plot work distributions
    print("--> plotting work distrib")
    
    accumulated_forward = [cycle[-1] for cycle in forward_work_offset]
    accumulated_reverse = [-cycle[-1] for cycle in reverse_work_offset]
    sns.distplot(accumulated_forward, color=CB_color_cycle[0], label='forward')
    sns.distplot(accumulated_reverse, color=CB_color_cycle[1], label='reverse')
    plt.axvline(dg)
    plt.axvline(dg + ddg, linestyle='dashed')
    plt.axvline(dg - ddg, linestyle='dashed')
    plt.xlabel("work (kT)")
    plt.ylabel("p(w)")
    plt.title(f"{title} {phase}")
    plt.legend(loc='best')
    if save:
        if output_dir is not None:
            plt.savefig(os.path.join(output_dir, f"{mutation}_{phase}_work_dist.png"), dpi=500)
            print(f"--> saved to: {os.path.join(output_dir, f'{mutation}_{phase}_work_dist.png')}")
        else:
            print("--> No output_dir specified!")
    else:
        plt.show()
    plt.clf()

In [21]:
# ntrk1 mutations
ntrk1_mutations = {
    'regorafenib': ['G595R', 'G667C'],
    'sorafenib': ['G595R', 'G667C'],
    }

In [28]:
base_data_path = '/data/chodera/glassw/kinoml/NTRK/1_improved_pipeline_Feb2021/run_noneq/NTRK1/'
base_output_dir = '/home/glassw/GITHUB/study-ntrk-resistance/notebooks/perses/perses_analysis_output/'
ntrk = 'NTRK1'

ntrk1_df = {
    'regorafenib': {},
    'sorafenib': {},
    }

for tki in ntrk1_mutations:

    for mutation in ntrk1_mutations[tki]:

        # Load and combine arrays
        forward_complex_arrays = []
        reverse_complex_arrays = []
        forward_apo_arrays = []
        reverse_apo_arrays = []

        for j in tqdm_notebook.tqdm(range(100)):

            forward_complex_path = f'{base_data_path}{mutation}/NTRK1_{tki}_{mutation}_complex_{j}_forward.npy'
            reverse_complex_path = f'{base_data_path}{mutation}/NTRK1_{tki}_{mutation}_complex_{j}_reverse.npy'
            forward_apo_path = f'{base_data_path}{mutation}/NTRK1_{tki}_{mutation}_apo_{j}_forward.npy'
            reverse_apo_path = f'{base_data_path}{mutation}/NTRK1_{tki}_{mutation}_apo_{j}_reverse.npy'

            if os.path.exists(forward_complex_path):
                with open(forward_complex_path, 'rb') as f:
                    loaded_np_array = np.load(f)
                    if not np.isnan(loaded_np_array.sum()):
                        forward_complex_arrays.append(loaded_np_array)

            if os.path.exists(reverse_complex_path):
                with open(reverse_complex_path, 'rb') as f:
                    loaded_np_array = np.load(f)
                    if not np.isnan(loaded_np_array.sum()):
                        reverse_complex_arrays.append(loaded_np_array)

            if os.path.exists(forward_apo_path):
                with open(forward_apo_path, 'rb') as f:
                    loaded_np_array = np.load(f)
                    if not np.isnan(loaded_np_array.sum()):
                        forward_apo_arrays.append(loaded_np_array)

            if os.path.exists(reverse_apo_path):
                with open(reverse_apo_path, 'rb') as f:
                    loaded_np_array = np.load(f)
                    if not np.isnan(loaded_np_array.sum()):
                        reverse_apo_arrays.append(loaded_np_array)
        
        if tki == 'regorafenib':
            if mutation == 'G667C':
                print(forward_complex_arrays)
                print(reverse_complex_arrays)

        if forward_complex_arrays and reverse_complex_arrays and forward_apo_arrays and reverse_apo_arrays:

            forward_complex_combined = np.concatenate(forward_complex_arrays)
            forward_complex_accumulated = np.array([cycle[-1] - cycle[0] for cycle in forward_complex_combined]) # compute this separately bc the last value of the subsampled array is diff than the actual last sample
            forward_complex_combined = np.array([cycle for cycle in forward_complex_combined])
            print(forward_complex_combined.shape)

            reverse_complex_combined = np.concatenate(reverse_complex_arrays)
            reverse_complex_accumulated = np.array([cycle[-1] - cycle[0] for cycle in reverse_complex_combined])
            reverse_complex_combined = np.array([cycle for cycle in reverse_complex_combined])

            forward_apo_combined = np.concatenate(forward_apo_arrays)
            forward_apo_accumulated = np.array([cycle[-1] - cycle[0] for cycle in forward_apo_combined])
            forward_apo_combined = np.array([cycle for cycle in forward_apo_combined])
            print(forward_apo_combined.shape)

            reverse_apo_combined = np.concatenate(reverse_apo_arrays)
            reverse_apo_accumulated = np.array([cycle[-1] - cycle[0] for cycle in reverse_apo_combined])
            reverse_apo_combined = np.array([cycle for cycle in reverse_apo_combined])


            # Analyse

            ## complex
            forward_complex_work_offset, reverse_complex_work_offset = subtract_offset(forward_complex_combined,
                                                                                    reverse_complex_combined)

            complex_dg, complex_ddg = analyse(forward_complex_accumulated,
                                            reverse_complex_accumulated)

            ## apo
            forward_apo_work_offset, reverse_apo_work_offset = subtract_offset(forward_apo_combined,
                                                                            reverse_apo_combined)

            apo_dg, apo_ddg = analyse(forward_apo_accumulated, reverse_apo_accumulated)

            ## make the output directories
            if not os.path.exists(f'{base_output_dir}{ntrk}/{tki}'):
                os.makedirs(f'{base_output_dir}{ntrk}/{tki}')

            ## plot the work trajectories and distibutions
            complex_plot = plot_works(forward_complex_work_offset,
                                    reverse_complex_work_offset,
                                    complex_dg,
                                    complex_ddg,
                                    phase='complex',
                                    mutation=mutation,
                                    title=f'{ntrk.upper()}-{tki} {mutation}',
                                    save=True,
                                    output_dir=f'{base_output_dir}{ntrk}/{tki}')

            apo_plot = plot_works(forward_apo_work_offset,
                                reverse_apo_work_offset,
                                apo_dg,
                                apo_ddg,
                                phase='apo',
                                mutation=mutation,
                                title=f'{ntrk.upper()}-{tki} {mutation}',
                                save=True,
                                output_dir=f'{base_output_dir}{ntrk}/{tki}')

            ## Get binding dg and ddg
            binding_dg = complex_dg - apo_dg
            binding_ddg = (apo_ddg**2 + complex_ddg**2)**0.5
            ntrk1_df[tki][mutation] = [binding_dg, binding_ddg]
            print(f"--> complex_dg: {complex_dg}")
            print(f"--> apo dg: {apo_dg}")

        else:
            print(f"--> dir {mutation} has at least one phase without data" )

  0%|          | 0/100 [00:00<?, ?it/s]

(100, 1251)
(100, 1251)
--> subtracting offset
--> computing dg, ddg
--> subtracting offset
--> computing dg, ddg
--> plotting work trajs
--> saved to: /home/glassw/GITHUB/study-ntrk-resistance/notebooks/perses/perses_analysis_output/NTRK1/regorafenib/G595R_complex_work_traj.png
--> plotting work distrib
--> saved to: /home/glassw/GITHUB/study-ntrk-resistance/notebooks/perses/perses_analysis_output/NTRK1/regorafenib/G595R_complex_work_dist.png
--> plotting work trajs
--> saved to: /home/glassw/GITHUB/study-ntrk-resistance/notebooks/perses/perses_analysis_output/NTRK1/regorafenib/G595R_apo_work_traj.png
--> plotting work distrib
--> saved to: /home/glassw/GITHUB/study-ntrk-resistance/notebooks/perses/perses_analysis_output/NTRK1/regorafenib/G595R_apo_work_dist.png
--> complex_dg: -267.86061485830936
--> apo dg: -275.80661623565385


  0%|          | 0/100 [00:00<?, ?it/s]

[array([[0.00000000e+00, 2.34404186e-05, 5.03188024e-03, ...,
        4.51662556e+01, 4.51915817e+01, 4.52186758e+01]]), array([[0.00000000e+00, 1.67151665e-05, 4.53081983e-03, ...,
        3.80368562e+01, 3.80590217e+01, 3.80812583e+01]]), array([[0.00000000e+00, 2.48411369e-05, 4.10495057e-03, ...,
        4.28194092e+01, 4.28422105e+01, 4.28657997e+01]]), array([[0.00000000e+00, 9.70983648e-06, 7.67175186e-05, ...,
        4.29039356e+01, 4.29316744e+01, 4.29600131e+01]]), array([[0.00000000e+00, 3.64086213e-05, 3.93740175e-03, ...,
        4.52942322e+01, 4.53197002e+01, 4.53442275e+01]]), array([[0.00000000e+00, 2.01785755e-05, 3.43391521e-03, ...,
        4.83524274e+01, 4.83805943e+01, 4.84063765e+01]]), array([[0.00000000e+00, 3.34842718e-05, 6.62881010e-03, ...,
        4.29748415e+01, 4.29955721e+01, 4.30149609e+01]]), array([[0.00000000e+00, 4.20069840e-06, 4.36881401e-03, ...,
        4.77252024e+01, 4.77486290e+01, 4.77741099e+01]]), array([[ 0.00000000e+00, -1.12106329e-0

--> computing dg, ddg
--> plotting work trajs
--> saved to: /home/glassw/GITHUB/study-ntrk-resistance/notebooks/perses/perses_analysis_output/NTRK1/regorafenib/G667C_complex_work_traj.png
--> plotting work distrib
--> saved to: /home/glassw/GITHUB/study-ntrk-resistance/notebooks/perses/perses_analysis_output/NTRK1/regorafenib/G667C_complex_work_dist.png
--> plotting work trajs
--> saved to: /home/glassw/GITHUB/study-ntrk-resistance/notebooks/perses/perses_analysis_output/NTRK1/regorafenib/G667C_apo_work_traj.png
--> plotting work distrib
--> saved to: /home/glassw/GITHUB/study-ntrk-resistance/notebooks/perses/perses_analysis_output/NTRK1/regorafenib/G667C_apo_work_dist.png
--> complex_dg: 37.09640015197215
--> apo dg: 27.003572983945507


  0%|          | 0/100 [00:00<?, ?it/s]

(100, 1251)
(100, 1251)
--> subtracting offset
--> computing dg, ddg
--> subtracting offset
--> computing dg, ddg
--> plotting work trajs
--> saved to: /home/glassw/GITHUB/study-ntrk-resistance/notebooks/perses/perses_analysis_output/NTRK1/sorafenib/G595R_complex_work_traj.png
--> plotting work distrib
--> saved to: /home/glassw/GITHUB/study-ntrk-resistance/notebooks/perses/perses_analysis_output/NTRK1/sorafenib/G595R_complex_work_dist.png
--> plotting work trajs
--> saved to: /home/glassw/GITHUB/study-ntrk-resistance/notebooks/perses/perses_analysis_output/NTRK1/sorafenib/G595R_apo_work_traj.png
--> plotting work distrib
--> saved to: /home/glassw/GITHUB/study-ntrk-resistance/notebooks/perses/perses_analysis_output/NTRK1/sorafenib/G595R_apo_work_dist.png
--> complex_dg: -269.0461555171649
--> apo dg: -253.50065463222236


  0%|          | 0/100 [00:00<?, ?it/s]

(99, 1251)
(100, 1251)
--> subtracting offset
--> computing dg, ddg
--> subtracting offset
--> computing dg, ddg
--> plotting work trajs
--> saved to: /home/glassw/GITHUB/study-ntrk-resistance/notebooks/perses/perses_analysis_output/NTRK1/sorafenib/G667C_complex_work_traj.png
--> plotting work distrib
--> saved to: /home/glassw/GITHUB/study-ntrk-resistance/notebooks/perses/perses_analysis_output/NTRK1/sorafenib/G667C_complex_work_dist.png
--> plotting work trajs
--> saved to: /home/glassw/GITHUB/study-ntrk-resistance/notebooks/perses/perses_analysis_output/NTRK1/sorafenib/G667C_apo_work_traj.png
--> plotting work distrib
--> saved to: /home/glassw/GITHUB/study-ntrk-resistance/notebooks/perses/perses_analysis_output/NTRK1/sorafenib/G667C_apo_work_dist.png
--> complex_dg: 29.201776838697853
--> apo dg: 27.447809083699074


<Figure size 432x288 with 0 Axes>

In [52]:
ntrk1_df

{'regorafenib': {'G595R': [7.946001377344487, 0.43286536539033804],
  'G667C': [10.092827168026641, 0.3828310490501222]},
 'sorafenib': {'G595R': [-15.545500884942527, 0.34497773810265314],
  'G667C': [1.7539677549987793, 0.30967900502556284]}}

In [42]:
# test_laro_df = save_csv(experimental_df_ntrk1, ntrk1_df['larotrectinib'], 'larotrectinib', 'NTRK1', output_path=output_path)

rego_df = pd.DataFrame(ntrk1_df['regorafenib']).T
rego_df2 = rego_df * KT_KCALMOL # convert simulated results from kT to kcal / mol

sora_df = pd.DataFrame(ntrk1_df['sorafenib']).T
sora_df2 = sora_df * KT_KCALMOL # convert simulated results from kT to kcal / mol

In [47]:
rego_df2.columns = ['DDG (kcal / mol)', 'dDDG (kcal / mol)']
rego_df2.insert(0, 'tki', 'regorafenib')
rego_df2

,tki,DDG (kcal / mol),dDDG (kcal / mol)
G595R,regorafenib,4.737098,0.258058
G667C,regorafenib,6.016953,0.228229


In [48]:
sora_df2.columns = ['DDG (kcal / mol)', 'dDDG (kcal / mol)']
sora_df2.insert(0, 'tki', 'sorafenib')
sora_df2

,tki,DDG (kcal / mol),dDDG (kcal / mol)
G595R,sorafenib,-9.267626,0.205662
G667C,sorafenib,1.045648,0.184619


In [50]:
save_df = pd.concat([rego_df2, sora_df2])

output_dir='/home/glassw/GITHUB/study-ntrk-resistance/notebooks/perses/perses_analysis_output/NTRK1/'

save_df.to_csv(f'{output_dir}prospective_rego_sora.csv')

In [51]:
save_df

,tki,DDG (kcal / mol),dDDG (kcal / mol)
G595R,regorafenib,4.737098,0.258058
G667C,regorafenib,6.016953,0.228229
G595R,sorafenib,-9.267626,0.205662
G667C,sorafenib,1.045648,0.184619
